# Tokenization
Machine learning approaches for natural language processing face the problem of representing long stretches of natural language (i.e. words, sentences, paragraphs, chapters, etc.) in a meaningful and computationally efficient way. A trivial approach is to split text on interpunction and whitespace, effectively selecting individual words. The downside of this approach is that semantically similar words are encoded differently. For example, 'small', 'smaller', and 'smallest', would all be encoded as different entities, forcing a model to learn any semantic similarity from data alone. An alternative approach would encode 'small' as one entity, and 'er', and 'est' as separate entities. The benefit of this 'subword' approach is that it is more straightforward to model semantic similarity, the downside is that it is not straightforward to identify an optimal subword selection scheme.

In this notebook we will explore the [byte pair encoding (BPE)](https://en.wikipedia.org/wiki/Byte_pair_encoding) algorithm for creating subword representations, a.k.a. tokens. Put simply, byte pair encoding iteratively merges the most frequent token pair into a new token, starting from the most simple tokens (e.g. letters), and continuing until the desired number of tokens is reached.

In [3]:
# All dependencies for the whole notebook
from collections import Counter
from itertools import chain
from tqdm.auto import trange
import json
from dataclasses import dataclass, field
from typing import Generator
from tokenizers import (
    decoders,
    models,
    trainers,
    Tokenizer,
)

## Data
We use the 'tiny shakespeare' dataset, which is a single textfile of ~1MB that contains all work of shakespeare. It has some interesting characteristics for which it is easy to evaluate if a model captures them well, such repeated newlines characters, all capital names, and specific interpunction used when describing theatrical plays/conversation.

In [4]:
# Download the tiny shakespeare dataset
!wget -nc https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

File ‘input.txt’ already there; not retrieving.



In [5]:
# Load the tiny shakespeare data and show the first 100 characters
with open('input.txt', 'r') as fh:
    data = fh.read()

data[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

## Naive 'tokens'
Probably the simplest tokenization strategy is to assign integers to individual characters in a given dataset. We'll implement this strategy below with a few lines of code. We create two dictionaries that function as lookup table: one encoding characters to integers, and one decoding integers back to characters. To apply our lookup tables we use a list comprehension.

In [6]:
# Calling 'set' on our data returns all individual characters, which are then lexicographically sorted
chars = sorted(set(data))
# Calling 'enumerate' returns the original iterator and increasing integers, which we'll use as tokens
char_to_token = {char:token for token,char in enumerate(chars)}
# Reverse the mapping to be able to decode
token_to_char = {token:char for char,token in char_to_token.items()}

some_text = data[:30]
print(f'{some_text = }')

# Encode the first 30 characters of the tiny shakespeare dataset
tokens = [char_to_token[c] for c in some_text]
print(f'{tokens = }')

# Check that we retrieve our original text when decoding
chars = [token_to_char[t] for t in tokens]
print(f'{chars = }')

some_text = 'First Citizen:\nBefore we proce'
tokens = [18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 14, 43, 44, 53, 56, 43, 1, 61, 43, 1, 54, 56, 53, 41, 43]
chars = ['F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n', 'B', 'e', 'f', 'o', 'r', 'e', ' ', 'w', 'e', ' ', 'p', 'r', 'o', 'c', 'e']


Below we add a bit more functionality and structure to the idea presented above. This allows us to more efficiently use our mappings as tokenizer, pass the tokenizer around more easily, and aligns with code conventions used in many of-the-shelf tokenizer libraries. The main elements are the same as above, e.g. the 'encoding_dict' is named 'char_to_token' in the example above, etc. Training (i.e. enumerating characters in a dataset) can be done with the `train` method, going from a string to a list of tokens is done with the `encode` method, going from a list of tokens back to a string is done with the `decode` method. 

In [7]:
class CharacterTokenizer:
    """Character level tokenizer that enumerates unique characters in a training text"""
    def __init__(self, encoding_dict: dict[str, int]=None):
        if encoding_dict is None:
            self.encoding_dict = dict()
        else:
            self.encoding_dict = encoding_dict

    def __repr__(self):
        return f'CharacterTokenizer(vocab_size={self.vocab_size})'

    @property
    def decoding_dict(self) -> dict[int, str]:
        """Decoding dict is implemented as property to automatically sync with changed encoding dict"""
        return {token:char for char,token in self.encoding_dict.items()}

    @property
    def vocab_size(self) -> int:
        return len(self.encoding_dict)

    def get_vocab(self) -> dict[str, int]:
        return self.encoding_dict

    def train(self, data: str) -> None:
        """Train on a piece of text by enumerating unique characters"""
        chars = sorted(set(data))
        self.encoding_dict = {char:token for token,char in enumerate(chars)}

    def encode(self, data: str) -> list[int]:
        """Convert text to tokens"""
        return [self.encoding_dict.get(char, -1) for char in data]

    def decode(self, tokens: list[int]) -> str:
        """Convert tokens to text"""
        return ''.join(self.decoding_dict.get(token, '<unk>') for token in tokens)

# Initialize an empty NaiveTokenizer
tokenizer = CharacterTokenizer()
print(f'Untrained tokenizer: {tokenizer}')

# 'Train' on tiny shakespeare
tokenizer.train(data)
print(f'Trained tokenizer: {tokenizer}')

# Encode a string that is not in the training data
tokens = tokenizer.encode('Hi how are you')
print(f'{tokens = }')

# Decode the encoding
print(tokenizer.decode(tokens))

Untrained tokenizer: CharacterTokenizer(vocab_size=0)
Trained tokenizer: CharacterTokenizer(vocab_size=65)
tokens = [20, 47, 1, 46, 53, 61, 1, 39, 56, 43, 1, 63, 53, 59]
Hi how are you


### Exercise 1
Investigate the vocabulary of the trained tokenizer by printing `tokenizer.get_vocab()`. Can you come up with a string that cannot be effectively encoded by our naive tokenizer? What happens to this string? How would you circumvent this issue?


## Byte Pair Encoding (BPE)

#### Converting to and from bytes
Apart from not being able to process unseen characters, tokenizing by enumerating characters in a training text has another issue: different training datasets can assign different tokens to the same character. The most common solution to these problems is to encode characters using [Unicode](https://en.wikipedia.org/wiki/Unicode) codepoints, specifically [UTF-8](https://en.wikipedia.org/wiki/UTF-8). Without going into too much detail, UTF-8 uses up to 4 bytes to encode individual characters, where every codepoint (i.e. byte or byte sequence) can be interpreted as an integer. The byte pair encoding algorithm iteratively merges these unicode codepoints.

In python, converting individual characters to unicode codepoints and back can be done with the built in `ord` and `chr` functions respectively.

In [8]:
# Single character to unicode
ord('H')

72

In [9]:
# Single unicode codepoint to character
chr(64)

'@'

In [10]:
# Converting a string to unicode and back
some_text = 'Deep learning is awesome'

unicode_codepoints = [ord(letter) for letter in some_text]
print(f'{unicode_codepoints = }')

characters = [chr(codepoint) for codepoint in unicode_codepoints]
print(f'{characters = }')

unicode_codepoints = [68, 101, 101, 112, 32, 108, 101, 97, 114, 110, 105, 110, 103, 32, 105, 115, 32, 97, 119, 101, 115, 111, 109, 101]
characters = ['D', 'e', 'e', 'p', ' ', 'l', 'e', 'a', 'r', 'n', 'i', 'n', 'g', ' ', 'i', 's', ' ', 'a', 'w', 'e', 's', 'o', 'm', 'e']


The same principles outlined above can be applied to multi-character strings with a slightly different syntax:

In [11]:
some_text = 'Deep learning is awesome'

# Using the 'encode' method on a string converts to bytes, note the leading 'b' when printing
text_bytes = some_text.encode('utf-8')
print(f'{text_bytes = }')

# The list constructor iterates over the bytes, automatically converting to integers
tokens = list(text_bytes)
print(f'{tokens = }')

# Turning a list of integers into bytes and subsequently 'decoding' into text
reconstructed_text = bytes(tokens).decode('utf-8')
print(f'{reconstructed_text = }')

text_bytes = b'Deep learning is awesome'
tokens = [68, 101, 101, 112, 32, 108, 101, 97, 114, 110, 105, 110, 103, 32, 105, 115, 32, 97, 119, 101, 115, 111, 109, 101]
reconstructed_text = 'Deep learning is awesome'


#### Counting pairs
Byte pair encoding iteratively merges the most frequent pairs of bytes. We use some built in python functionality to count pairs in an iterator (the example uses characters, BPE  uses bytes).

In [12]:
example_sequence = 'A text with some repetition somesome reprepreprepetition'

# zip together original and shifted sequence
pairs = zip(example_sequence[:-1], example_sequence[1:])

# count using inbuilt counter from collections module (part of python standard lib)
pair_counts = Counter(pairs)

# select most common pair
most_common_pair = pair_counts.most_common(1)
print(f'{most_common_pair = }')

most_common_pair = [(('r', 'e'), 5)]


#### BPE implementation
Below we implement a function to merge token pairs and some functionality to train the tokenizer, encode strings, decode tokens, and save and load trained models.

In [13]:
def merge_tokens(tokens: list[int], token_pair: tuple[int,int], new_token: int) -> list[int]:
    """Takes a list of tokens and replaces every occurence of token_pair with new_token"""
    new_tokens = []
    i = 0
    # Iterate in a while loop because we want to jump ahead two steps sometimes
    while i < len(tokens):
        token = tokens[i]
        # Edge case: final individual token
        if i == len(tokens) - 1:
            new_tokens.append(token)
            break
        # Look ahead one token to find a token pair
        next_token = tokens[i+1]
        # On match we should jump ahead two tokens to skip the original pair
        if token_pair == (token, next_token):
            new_tokens.append(new_token)
            i += 2
        else:
            new_tokens.append(token)
            i += 1
    return new_tokens

class BytePairEncoder:
    """Bytepair encoder with a base vocabulary of the first 256 utf-8 codepoints (this captures all 'normal' alphanumeric characters)"""
    def __init__(self, merges: dict[int, tuple[int, int]]=None):
        if merges is None:
            self.merges = dict()
        else:
            self.merges = merges

    def __repr__(self) -> str:
        vocab_size = self.vocab_size
        n_merges = len(self.merges)
        return f'BytePairEncoder({vocab_size=} {n_merges=})'

    @property
    def vocab_size(self) -> int:
        return len(self.get_vocab())

    def get_vocab(self) -> dict[int, str]:
        # Base vocabulary of first 256 utf-8 characters
        base_vocab = {chr(token): token for token in range(128)}
        # Additional vocabulary is determined by trained merges
        merge_vocab = {self.decode([token]): token for token in self.merges}
        # Total vocabulary is the union of base and merge vocabs
        vocab = base_vocab | merge_vocab
        return vocab

    def _get_parent_tokens(self, token: int) -> Generator[int, None, None]:
        """Recursively identify whether a token is made up of parent tokens"""
        if token not in self.merges:
            yield token
            return
        for pair_token in self.merges[token]:
            yield from self._get_parent_tokens(pair_token)

    def train(self, input: str, vocab_size: int = 512) -> None:
        """Training proceeds by iteratively merging the most frequent token pair until the desired number of tokens is reached"""
        assert vocab_size > 128, f'Invalid vocab_size: {vocab_size}, must be larger than 256'
        tokens = list(input.encode('utf-8'))
        num_merges = vocab_size - 128
        for i in trange(num_merges):
            pair_counts = Counter(zip(tokens[:-1], tokens[1:]))
            merge_pair = pair_counts.most_common(1)[0][0]
            new_token = 128 + i
            self.merges[new_token] = merge_pair
            tokens = merge_tokens(tokens, merge_pair, new_token)

    def encode(self, input: str) -> list[int]:
        """Convert text to tokens"""
        tokens = list(input.encode('utf-8'))
        for new_token, merge_pair in self.merges.items():
            tokens = merge_tokens(tokens, merge_pair, new_token)
        return tokens

    def decode(self, tokens: list[int]) -> str:
        """Convert tokens to text"""
        decoded_tokens = chain.from_iterable(map(self._get_parent_tokens, tokens))
        return bytes(decoded_tokens).decode('utf-8', errors='replace')

    def save(self, prefix: str) -> None:
        """Save a trained model"""
        with open(f'{prefix}.vocab', 'w') as fh:
            json.dump(self.get_vocab(), fh)
        with open(f'{prefix}.model', 'w') as fh:
            json.dump(self.merges, fh)

    @classmethod
    def load(cls, model_filename: str) -> 'BytePairEncoder':
        """Load a pretrained model from a .model file"""
        assert model_filename.endswith('.model'), f'{model_filename} is not a valid model file, must end with .model'
        with open(model_filename, 'r') as fh:
            merges = json.load(fh)
        # The json fileformat does not accept integers as dict keys, and does not have tuples
        sanitized_merges = {int(k):tuple(v) for k,v in merges.items()}
        return cls(sanitized_merges)

# Show base vocabulary without training
BytePairEncoder().get_vocab()

{'\x00': 0,
 '\x01': 1,
 '\x02': 2,
 '\x03': 3,
 '\x04': 4,
 '\x05': 5,
 '\x06': 6,
 '\x07': 7,
 '\x08': 8,
 '\t': 9,
 '\n': 10,
 '\x0b': 11,
 '\x0c': 12,
 '\r': 13,
 '\x0e': 14,
 '\x0f': 15,
 '\x10': 16,
 '\x11': 17,
 '\x12': 18,
 '\x13': 19,
 '\x14': 20,
 '\x15': 21,
 '\x16': 22,
 '\x17': 23,
 '\x18': 24,
 '\x19': 25,
 '\x1a': 26,
 '\x1b': 27,
 '\x1c': 28,
 '\x1d': 29,
 '\x1e': 30,
 '\x1f': 31,
 ' ': 32,
 '!': 33,
 '"': 34,
 '#': 35,
 '$': 36,
 '%': 37,
 '&': 38,
 "'": 39,
 '(': 40,
 ')': 41,
 '*': 42,
 '+': 43,
 ',': 44,
 '-': 45,
 '.': 46,
 '/': 47,
 '0': 48,
 '1': 49,
 '2': 50,
 '3': 51,
 '4': 52,
 '5': 53,
 '6': 54,
 '7': 55,
 '8': 56,
 '9': 57,
 ':': 58,
 ';': 59,
 '<': 60,
 '=': 61,
 '>': 62,
 '?': 63,
 '@': 64,
 'A': 65,
 'B': 66,
 'C': 67,
 'D': 68,
 'E': 69,
 'F': 70,
 'G': 71,
 'H': 72,
 'I': 73,
 'J': 74,
 'K': 75,
 'L': 76,
 'M': 77,
 'N': 78,
 'O': 79,
 'P': 80,
 'Q': 81,
 'R': 82,
 'S': 83,
 'T': 84,
 'U': 85,
 'V': 86,
 'W': 87,
 'X': 88,
 'Y': 89,
 'Z': 90,
 '[': 91,


### Exercise 2
Train a byte pair encoder on the tiny shakespeare dataset with a vocab_size of 256 and inspect the vocabulary. Can you identify tokens that encode some semantically meaningful identity?

In [16]:
# Train and save a BPE tokenizer
bpe = BytePairEncoder()
bpe.train(data)
bpe.get_vocab()

  0%|          | 0/384 [00:00<?, ?it/s]

{'\x00': 0,
 '\x01': 1,
 '\x02': 2,
 '\x03': 3,
 '\x04': 4,
 '\x05': 5,
 '\x06': 6,
 '\x07': 7,
 '\x08': 8,
 '\t': 9,
 '\n': 10,
 '\x0b': 11,
 '\x0c': 12,
 '\r': 13,
 '\x0e': 14,
 '\x0f': 15,
 '\x10': 16,
 '\x11': 17,
 '\x12': 18,
 '\x13': 19,
 '\x14': 20,
 '\x15': 21,
 '\x16': 22,
 '\x17': 23,
 '\x18': 24,
 '\x19': 25,
 '\x1a': 26,
 '\x1b': 27,
 '\x1c': 28,
 '\x1d': 29,
 '\x1e': 30,
 '\x1f': 31,
 ' ': 32,
 '!': 33,
 '"': 34,
 '#': 35,
 '$': 36,
 '%': 37,
 '&': 38,
 "'": 39,
 '(': 40,
 ')': 41,
 '*': 42,
 '+': 43,
 ',': 44,
 '-': 45,
 '.': 46,
 '/': 47,
 '0': 48,
 '1': 49,
 '2': 50,
 '3': 51,
 '4': 52,
 '5': 53,
 '6': 54,
 '7': 55,
 '8': 56,
 '9': 57,
 ':': 58,
 ';': 59,
 '<': 60,
 '=': 61,
 '>': 62,
 '?': 63,
 '@': 64,
 'A': 65,
 'B': 66,
 'C': 67,
 'D': 68,
 'E': 69,
 'F': 70,
 'G': 71,
 'H': 72,
 'I': 73,
 'J': 74,
 'K': 75,
 'L': 76,
 'M': 77,
 'N': 78,
 'O': 79,
 'P': 80,
 'Q': 81,
 'R': 82,
 'S': 83,
 'T': 84,
 'U': 85,
 'V': 86,
 'W': 87,
 'X': 88,
 'Y': 89,
 'Z': 90,
 '[': 91,


## Optimized BPE using the Huggingface transformers library
As you can imagine, our python implementation is not optimized to be fast. Several optimized tokenizers are commonly used, most of which have python bindings for ease of use. Below we will reproduce the configuration of our python BPE tokenizer using the [tokenizers section of the Huggingface transformers library](https://huggingface.co/docs/transformers/en/fast_tokenizers). This allows us to train larger vocabularies in a shorter amount of time.

> _Note:_ [Huggingface](https://huggingface.co/) is a large online community for sharing machine learning datasets, models, and applications. Here we use the BPE tokenizer, later we will also use pretrained models.  

In [17]:
# Specify a BPE tokenizer using Huggingface's tokenizers
tokenizer = Tokenizer(models.BPE(byte_fallback=True))
trainer = trainers.BpeTrainer(
    initial_alphabet=[chr(i) for i in range(128)], # same base vocabulary as our python implementation
    vocab_size=256
)
tokenizer.decoder = decoders.ByteLevel()

# Train on shakespeare
tokenizer.train(["input.txt"], trainer=trainer)

Below we do some quick validation that the Huggingface tokenizer and our python tokenizer do the same.

In [18]:
# Check whether our test string is turned into the same tokens
test_string = 'Hi how are you 1234'

huggingface_tokens = tokenizer.encode("Hi how are you 1234").tokens
print(f'{huggingface_tokens = }')

our_tokens = [bpe.decode([i]) for i in bpe.encode('Hi how are you 1234')]
print(f'{our_tokens         = }')

huggingface_tokens = ['H', 'i', ' h', 'ow', ' ', 'ar', 'e ', 'you', ' ', '1', '2', '3', '4']
our_tokens         = ['H', 'i', ' h', 'ow', ' ', 'are ', 'you ', '1', '2', '3', '4']


In [ ]:
# Check whether the same tokens use the same ids
huggingface_token_ids = tokenizer.encode(test_string).ids
print(f'{huggingface_token_ids = }')

our_token_ids = bpe.encode(test_string)
print(f'{our_token_ids         = }')

Note that whereas the Huggingface tokenizer identifies the same character combinations, it assigns different identifiers for several tokens! After some debugging, it seems that the Huggingface tokenizer is inconsistent in how it handles double newline characters (i.e. '\n\n'), and there has been some discussion on this feature for [similar tokenizers](https://discuss.huggingface.co/t/gpt2tokenizer-tokenizer-handling-n-n-differently-in-different-settings/57289). For all our practical purposes this currently does not matter too much.

### Exercise 3
Train a BPE encoder using the Huggingface tokenizer with a vocab_size of 1024 and inspect the vocabulary. Similar as in exercise 2: do you see tokens that encode meaningful semantics (just like in our python implementation, you can use `tokenizer.get_vocab()` to inspect the learned vocabulary)? What happens if you increase the vocab_size even further? Is there a limit to the vocab_size?

## Answers

### Exercise 1
The only number in the vocabulary is '3', so any other number would work, as would signs like for example a #. These characters get a token -1, '< unk >' (for "unknown"). A solution is to use a base vocabulary of all known or relevant single characters.

### Exercise 2
Code below trains, saves, and loads a BPE tokenizer and prints the learned vocabulary.

In [ ]:
# Train and save a BPE tokenizer
bpe = BytePairEncoder()
bpe.train(data, vocab_size=256)
bpe.save(f'shakespeare_{bpe.vocab_size}')
bpe = BytePairEncoder.load('./shakespeare_256.model')
bpe.get_vocab()

There are several words and word fragments, such as 'the', 'and', 'or', 'with', 'have', and 'your'. Interestingly, 'thou' is also in the vocab, indicating the typical language of Shakespeare's texts.

### Exercise 3
The larger the vocabulary, the more word-like tokens become, up to a point where multiple words get merged into one token (e.g. a vocab_size of 20000 gives 'You shall not ' as one of the tokens). The upper limit is data dependent and occurs when no further merges can happen; for a large dataset this limit will be high.